### Initiate Spark 

control a Spark Application through a driver process called SparkSession

1) on console: ```spark```

2) on jupyternotebook: ```jupyter notebook``` then in a cell run


In [67]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

### Spark UI 

```http://localhost:4040/jobs/```

#todo to 5 

## Ch5: Basic Structured Operations

### Dataframes Schemas

1) schema-on-read (autodetect)

2) defined explicitly

In [68]:
# to load and check the schema  without schema(myManulaSchema)
spark.read.format('json').load('./data/flight-data/json/2015-summary.json').schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [69]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
  StructField("DEST_COUNTRY_NAME", StringType(), True),
  StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
  StructField("count", LongType(), False, metadata={"hello":"world"})
])
flights_df = spark.read.format("json").schema(myManualSchema)\
  .load("./data/flight-data/json/2015-summary.json")

flights_df .schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

A schema is a ```StructType``` build by ```StructField``` made of:

    1) nameColumn
    
    2) typeColumn
    
    3) Nullable
    
    4) metadata (optional)

To check the schema 

In [53]:
flights_df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



### Columns

The book combines the Scala and PySpark API's.

In Scala / Java API, ```df.col("column_name")```,```df.col('column_name)```,```df("column_name")``` or  ```df.apply("column_name")``` return the Column.

Whereas in pyspark use the below to get the column from DF.

```df.colName```
```df["colName"]```

<b>HOWEVER </b>, if using  ```select``` it is also possible to use ```col("column_name")```

In [57]:
from pyspark.sql.functions import col, column

# df("someColumnName")
flights_df["DEST_COUNTRY_NAME"]
flights_df.DEST_COUNTRY_NAME


Column<'DEST_COUNTRY_NAME'>

In [58]:
flights_df.columns #column property to access columns 

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

## Expressions 

an expression parses transformations and column references from a string 

In [43]:
from pyspark.sql.functions import expr

df = spark.range(500).toDF("number")
df.select(df["number"] + 10).take(3)
 
df.select(expr("(((number + 5) * 200) - 6) < 5")).take(3)

[Row(((((number + 5) * 200) - 6) < 5)=False),
 Row(((((number + 5) * 200) - 6) < 5)=False),
 Row(((((number + 5) * 200) - 6) < 5)=False)]

### Rows
Each row is a single record, represented as an object of type ```Row```. To manipulate an object of type ```Row``` use a column expression (previous paragraph). Internally represent arrays of bytes.

In [46]:
df.first() # an example to check a type Row is printing

Row(number=0)

#### Create Rows
1) manually instanciatin an object ```Row``` (values in the same order and type as the schema of the df to which you have to append them


In [48]:
from pyspark.sql import Row
myRow = Row("Hello", None, 1, False)

In [49]:
myRow[0] # to access the value

'Hello'

### DataFrame

#### Creating df
1) from a file / raw data sources ```spark.read.format('format').source('path/to/data')```
2) from a set of rows 

In [50]:
from pyspark import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
  StructField("Welcome", StringType(), True),
  StructField("None", StringType(), True),
  StructField("number", LongType(), False, metadata={"hello":"world"})
])

myRow = Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow], myManualSchema)
myDf.show()

+-------+----+------+
|Welcome|None|number|
+-------+----+------+
|  Hello|null|     1|
+-------+----+------+



#### Transforming a Df
To transform a Df we can only manipulate columns (rows singularly are not accessible) and we can use 
1) ```select``` method

2) ```selectExpr``` method

3) ```import pyspark.sql.functions``` package

#### Transforming using SELECT

In [61]:
flights_df.select("DEST_COUNTRY_NAME").show(2) # singular selection

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [62]:
flights_df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2) #multiple selection


+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

In [76]:
from pyspark.sql.functions import expr, column, col 
flights_df.select(expr("DEST_COUNTRY_NAME"),
                 col("ORIGIN_COUNTRY_NAME"),
#                  column("count")               # column is not working 
                 ).show(5)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
|    United States|            Ireland|
|            Egypt|      United States|
|    United States|              India|
+-----------------+-------------------+
only showing top 5 rows



<b> NOT TRUE common mistake </b>: use a mix of column Objects and strings, does not give an error

In [79]:
from pyspark.sql.functions import expr, column, col 
flights_df.select(
                 col("ORIGIN_COUNTRY_NAME"),"DEST_COUNTRY_NAME"
                 ).show(5)

+-------------------+-----------------+
|ORIGIN_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-------------------+-----------------+
|            Romania|    United States|
|            Croatia|    United States|
|            Ireland|    United States|
|      United States|            Egypt|
|              India|    United States|
+-------------------+-----------------+
only showing top 5 rows



#### Rename columns 

In [83]:
flights_df.select(expr("ORIGIN_COUNTRY_NAME AS origin")).show(2)
flights_df.select(expr("ORIGIN_COUNTRY_NAME").alias("origin2")).show(2) #### NB the alias is INSIDE select

+-------+
| origin|
+-------+
|Romania|
|Croatia|
+-------+
only showing top 2 rows

+-------+
|origin2|
+-------+
|Romania|
|Croatia|
+-------+
only showing top 2 rows



#### Transforming using .selectExpr()
Because ```select``` and ```expr``` is a commone pattern --> short hand ```selectExpr```

In [87]:
flights_df.selectExpr("ORIGIN_COUNTRY_NAME AS origin").show(2)

+-------+
| origin|
+-------+
|Romania|
|Croatia|
+-------+
only showing top 2 rows



In [88]:
flights_df.selectExpr("*","ORIGIN_COUNTRY_NAME AS origin", "ORIGIN_COUNTRY_NAME = DEST_COUNTRY_NAME as withInCountry").show(2)

+-----------------+-------------------+-----+-------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count| origin|withInCountry|
+-----------------+-------------------+-----+-------+-------------+
|    United States|            Romania|   15|Romania|        false|
|    United States|            Croatia|    1|Croatia|        false|
+-----------------+-------------------+-----+-------+-------------+
only showing top 2 rows



###  Literals
```lit``` is used to pass explicit values into Spark that are just a value. Need to be imported 

In [97]:
from pyspark.sql.functions import lit 
flights_df.select(expr("*"), lit(True)).show(2) # lit is OUTSIDE expr()
flights_df.select(expr("*"), lit(True).alias("True value?")).show(2) # lit is OUTSIDE expr()

#NB a difference with SCALA is that .alias() in Python is like .as() in Scala

+-----------------+-------------------+-----+----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|true|
+-----------------+-------------------+-----+----+
|    United States|            Romania|   15|true|
|    United States|            Croatia|    1|true|
+-----------------+-------------------+-----+----+
only showing top 2 rows

+-----------------+-------------------+-----+-----------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|True value?|
+-----------------+-------------------+-----+-----------+
|    United States|            Romania|   15|       true|
|    United States|            Croatia|    1|       true|
+-----------------+-------------------+-----+-----------+
only showing top 2 rows



### Adding Columns with ```withColumns('column_name', value)``` 

In [98]:
flights_df.withColumn('numberOne', lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [99]:
# withColumn(column_name, expression)
flights_df.withColumn('withInCountry', expr("DEST_COUNTRY_NAME=ORIGIN_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withInCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



### Rename a column ```withColumnRenamed('old_name', 'new_name')```

In [101]:
new_flights_df= flights_df.withColumnRenamed("DEST_COUNTRY_NAME", 'destination')
new_flights_df.show(2)

+-------------+-------------------+-----+
|  destination|ORIGIN_COUNTRY_NAME|count|
+-------------+-------------------+-----+
|United States|            Romania|   15|
|United States|            Croatia|    1|
+-------------+-------------------+-----+
only showing top 2 rows



### Remove columns


In [103]:
new_flights_df=new_flights_df.drop('destination')
new_flights_df.show(2)

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Romania|   15|
|            Croatia|    1|
+-------------------+-----+
only showing top 2 rows



### Changing Column Type ```cast('type')```

In [106]:
new_flights_df.withColumn("StringNumber", col('count').cast('string')).show(2)
new_flights_df.withColumn("StringNumber", col('count').cast('string')).printSchema()

+-------------------+-----+------------+
|ORIGIN_COUNTRY_NAME|count|StringNumber|
+-------------------+-----+------------+
|            Romania|   15|          15|
|            Croatia|    1|           1|
+-------------------+-----+------------+
only showing top 2 rows

root
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)
 |-- StringNumber: string (nullable = true)



### Filtering ```where()``` or ```filter()```

In [107]:
new_flights_df.filter(col("count") < 2 ).show(2)

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Croatia|    1|
|          Singapore|    1|
+-------------------+-----+
only showing top 2 rows



In [111]:
new_flights_df.where(col("count") < 2 ).show(2)
new_flights_df.where(expr("count")< 2 ).show(2)
# new_flights_df.where("count"< 2 ).show(2) # NOT working is comapring strign and number 

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Croatia|    1|
|          Singapore|    1|
+-------------------+-----+
only showing top 2 rows

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Croatia|    1|
|          Singapore|    1|
+-------------------+-----+
only showing top 2 rows



In [113]:
new_flights_df.limit(3).show()

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Romania|   15|
|            Croatia|    1|
|            Ireland|  344|
+-------------------+-----+



###

###

###

###

In [41]:
df.select(expr("(((column('number') + 5) * 200) - 6) < 5")).take(3)

AnalysisException: Undefined function: 'column'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; line 1 pos 3